In [ ]:
import numpy as np
import logging
logging.basicConfig(level=logging.INFO, force=True)
from dataclasses import dataclass
from pathlib import Path
   
# teleop
from teleoperation_system.teleoperators import TeleoperationSystem, TeleoperationSystemConfig, TeleopManager

# robots
from teleoperation_system.robots import make_robot_from_config, BimanualRobotConfig
from teleoperation_system.robots.ur3 import UR3Config

# cameras
from teleoperation_system.cameras.realsense import RealSenseCameraConfig
from teleoperation_system.cameras.opencv import OpenCVCameraConfig

# dataset
from teleoperation_system.data_utils.lerobot_dataset import LeRobotDataset, DataRecordingManager
from teleoperation_system.data_utils.utils import hw_to_dataset_features

from teleoperation_system.utils import sanity_check_dataset_robot_compatibility

### Without dataset collection

In [ ]:
# CONFIGURATION
fps = 50

left_ip = "192.168.1.111"   # IP address for the left arm
right_ip = "192.168.1.110"  # IP address for the right arm

left_base_pose = np.array(
    [
        0.0002879959065467119,
        -1.5719226042376917,
        -1.5715907255755823,
        3.1422481536865234,
        -1.5707929770099085,
        512.0812350745701,
    ]
)

right_base_pose = np.array(
    [
        -0.001450840626851857,
        -1.5725587050067347,
        1.5713553428649902,
        0.0008134841918945312,
        1.5712484121322632,
        9.42502719560732,
    ]
)

teleoperation_system_config = TeleoperationSystemConfig()
teleoperation_sysytem = TeleoperationSystem(teleoperation_system_config)

robot_config = UR3Config(ip=right_ip, arm = "right", base_pose=right_base_pose)
robot = make_robot_from_config(robot_config)

In [ ]:
# Move each arm to its respective base pose
robot.connect()
robot.move_to_base_pose()

In [ ]:
with TeleopManager(teleop = teleoperation_sysytem, robot = robot, fps = fps) as manager:
    while True:
        manager.wait_for_episode_start()

        data_from_teleoperation_system = teleoperation_sysytem.get_data()
        if data_from_teleoperation_system is None:
            teleoperation_sysytem.is_data_timeout()
            continue
        teleoperation_sysytem.reset_watchdog() 
        
        if data_from_teleoperation_system is None:
            teleoperation_sysytem.is_data_timeout()
            continue
        teleoperation_sysytem.reset_watchdog() 

        action = robot.calculate_action(data_from_teleoperation_system)

        # send action to the robot
        robot.send_action(action)

### With dataset collection

In [ ]:
################################################################# Main configuration class ###############################################################
@dataclass
class RecordConfig:
    dataset_path: str | Path # Root directory where the dataset will be stored (e.g. 'dataset/path').
    record_data: bool = True
    resume: bool = True
    fps: int = 25 # Limit the frames per second.
    use_videos: bool = True # Encode images into videos to save space

home_dir = Path.home()
cfg = RecordConfig(dataset_path = home_dir /"lerobot_datasets"/"test")

###################################################################### Robot configuration ##################################################################
left_ip = "192.168.1.111"   # IP address for the left arm
right_ip = "192.168.1.110"  # IP address for the right arm

left_base_pose = np.array(
    [
        0.0002879959065467119,
        -1.5719226042376917,
        -1.5715907255755823,
        3.1422481536865234,
        -1.5707929770099085,
        512.0812350745701,
    ]
)

right_base_pose = np.array(
    [
        -0.001450840626851857,
        -1.5725587050067347,
        1.5713553428649902,
        0.0008134841918945312,
        1.5712484121322632,
        9.42502719560732,
    ]
)

cameras_config = {"ego_view": RealSenseCameraConfig(), "wrist_view": OpenCVCameraConfig(use_node=True, camera_node = "/dev/video6")}
robot_config = UR3Config(ip=right_ip, arm = "right", base_pose=right_base_pose, cameras = cameras_config)

teleoperation_system_config = TeleoperationSystemConfig()

In [ ]:
teleoperation_sysytem = TeleoperationSystem(teleoperation_system_config)
robot = make_robot_from_config(robot_config)

if cfg.record_data:
    action_features = hw_to_dataset_features(robot.action_features, "action", cfg.use_videos)
    obs_features = hw_to_dataset_features(robot.observation_features, "observation", cfg.use_videos)
    dataset_features = {**action_features, **obs_features}
    if cfg.resume:
        #  Resume from existing dataset
        dataset = LeRobotDataset(
            root=cfg.dataset_path,
        )
        sanity_check_dataset_robot_compatibility(dataset, robot, cfg.fps, dataset_features)
        
    else:
        # Create empty dataset
        dataset = LeRobotDataset(
            fps= cfg.fps,
            root=cfg.dataset_path,
            robot_type=robot.name,
            features=dataset_features,
            use_videos=cfg.use_videos,
        )
else:
    dataset = None

In [ ]:
# Move each arm to its respective base pose
robot.connect()
robot.move_to_base_pose()

task = "Take the plastic toys from the table and put them in the box."

In [ ]:
# use this line to delete last recorded episode
# dataset.delete_last_episode()

In [ ]:
with DataRecordingManager(dataset = dataset, robot = robot, teleop = teleoperation_sysytem, fps = cfg.fps) as data_recorder:
    while True:
        data_recorder.wait_for_episode_start()

        data_from_teleoperation_system = teleoperation_sysytem.get_data()
        if data_from_teleoperation_system is None:
            teleoperation_sysytem.is_data_timeout()
            continue
        teleoperation_sysytem.reset_watchdog() 

        action = robot.calculate_action(data_from_teleoperation_system)

        if cfg.record_data:
            data_recorder.record(data_from_teleoperation_system, action, task)

        robot.send_action(action)

### Bimanual manipulation

In [ ]:
################################################################# Main configuration class ###############################################################
@dataclass
class RecordConfig:
    # Root directory where the dataset will be stored (e.g. 'dataset/path').
    dataset_path: str | Path

    record_data: bool = True

    resume: bool = False
    # Limit the frames per second.
    fps: int = 20
    # Encode images into videos to save space
    use_videos: bool = True

home_dir = Path.home()
cfg = RecordConfig(dataset_path = home_dir /"lerobot_datasets"/"test")
###################################################################### Robot configuration ##################################################################
left_ip = "192.168.1.111"   # IP address for the left arm
right_ip = "192.168.1.110"  # IP address for the right arm

left_base_pose = np.array(
    [
        0.0002879959065467119,
        -1.5719226042376917,
        -1.5715907255755823,
        3.1422481536865234,
        -1.5707929770099085,
        512.0812350745701,
    ]
)

right_base_pose = np.array(
    [
        -0.001450840626851857,
        -1.5725587050067347,
        1.5713553428649902,
        0.0008134841918945312,
        1.5712484121322632,
        9.42502719560732,
    ]
)

cameras_config = {"ego_view": RealSenseCameraConfig(), "right_wrist_view": OpenCVCameraConfig(use_node=True, camera_node = "/dev/video6")}
left_arm_config = UR3Config(ip = left_ip, arm = "left", base_pose=left_base_pose, use_gripper=False, use_force_sensor=False)
right_arm_config = UR3Config(ip = right_ip, arm = "right", base_pose=right_base_pose)
robot_config = BimanualRobotConfig(left_arm_config=left_arm_config, right_arm_config=right_arm_config, cameras=cameras_config)
teleoperation_system_config = TeleoperationSystemConfig()

In [ ]:
teleoperation_sysytem = TeleoperationSystem(teleoperation_system_config)
robot = make_robot_from_config(robot_config)

if cfg.record_data:
    action_features = hw_to_dataset_features(robot.action_features, "action", cfg.use_videos)
    obs_features = hw_to_dataset_features(robot.observation_features, "observation", cfg.use_videos)
    dataset_features = {**action_features, **obs_features}
    if cfg.resume:
        #  Resume from existing dataset
        dataset = LeRobotDataset(
            root=cfg.dataset_path,
        )
        sanity_check_dataset_robot_compatibility(dataset, robot, cfg.fps, dataset_features)
        
    else:
        # Create empty dataset
        dataset = LeRobotDataset(
            fps= cfg.fps,
            root=cfg.dataset_path,
            robot_type=robot.name,
            features=dataset_features,
            use_videos=cfg.use_videos,
        )
else:
    dataset = None

In [ ]:
# Move each arm to its respective base pose
robot.connect()
robot.move_to_base_pose()
task = "some new task"

In [ ]:
with DataRecordingManager(dataset = dataset, robot = robot) as data_recorder:
    while True:
        data_recorder.wait_for_episode_start()

        data_from_teleoperation_system = teleoperation_sysytem.get_data()
        if data_from_teleoperation_system is None:
            teleoperation_sysytem.is_data_timeout()
            continue
        teleoperation_sysytem.reset_watchdog() 

        action = robot.calculate_action(data_from_teleoperation_system)

        if cfg.record_data:
            data_recorder.record(data_from_teleoperation_system, action, task)

        robot.send_action(action)